In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.countplot(x='diagnosis',data=df)

#### Dataset seems to be balanced. No special handling required for sampling

In [ ]:
df.columns

#### There are 2 columns that are of no value : "id" and "Unnamed: 32". Drop them

In [ ]:
df.drop(columns=['Unnamed: 32', 'id'], inplace = True)

In [ ]:
df.columns

#### Map the diagnosis code to integer. Benign :0, Malignant:1

In [ ]:
res = {'B':0, 'M':1}
df['diagnosis'] = df['diagnosis'].map(res)

In [ ]:
df.columns

#### Choose the features that have 0.5 or more correlation to the result

In [ ]:
cor_target = abs(df.corr()['diagnosis'])
set_features = cor_target[cor_target > 0.5]
df = df[set_features.index]
df.head()

In [ ]:
df.info()

In [ ]:
df.corr()['diagnosis'][1:].sort_values().plot(kind='bar')

In [ ]:
X = df.drop('diagnosis',axis=1).values
y = df['diagnosis'].values

#### Split the data Train:validation:test = 60:20:20 ratio

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=101)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [ ]:
model = Sequential()


# input layer
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.5))

# hidden layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# hidden layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))


# output layer
model.add(Dense(units=1,activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x=X_train, 
          y=y_train, 
          epochs=200,
          batch_size = 16,
          validation_data=(X_val, y_val), verbose =1,
          callbacks=[early_stop]
          )

In [ ]:
plt.figure(1, figsize = (15,8)) 
plt.subplot(221)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
    
plt.subplot(222)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 

plt.show()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
predictions = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
print(classification_report(predictions, y_test))

In [ ]:
#confusion_matrix(y_test,predictions)
cm = confusion_matrix(predictions, y_test)
sns.heatmap(cm, annot=True, cmap='Blues_r')
